# Mod 2 Summative Project - Soccer Match DB Transfer and Analysis

In [1]:
from SqlConn import SqlConn
from MongoHandler import MongoHandler
from PandaHandler import PandaHandler
from RainData import RainData
from dark_sky_api_remy import api_key as api_key
import time
import numpy as np
import json
import requests
import pandas as pd
import datetime
from matplotlib.image import imread
import matplotlib.pyplot as plt
%matplotlib inline

### Making a connection to the SQL Database and getting back a Pandas DataFrame

In [2]:
sql = SqlConn('database.sqlite')

Connection status: Active


In [3]:
df = sql.matches_df([2011])
df.head()

Connection status: Active


,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1133,D2,2011,2011-07-15,Cottbus,Dresden,2,1,H
1,1167,D2,2011,2011-07-15,Greuther Furth,Ein Frankfurt,2,3,A
2,1551,D2,2011,2011-07-15,Frankfurt FSV,Union Berlin,1,1,D
3,1550,D2,2011,2011-07-16,Erzgebirge Aue,Aachen,1,0,H
4,1678,D2,2011,2011-07-16,St Pauli,Ingolstadt,2,0,H


In [4]:
sql.close_conn()

Closing connection
Connection status: Closed


In [5]:
df = PandaHandler.tot_home_goals_scored(df)
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,tot_home_goals
0,1133,D2,2011,2011-07-15,Cottbus,Dresden,2,1,H,18
1,1167,D2,2011,2011-07-15,Greuther Furth,Ein Frankfurt,2,3,A,46
2,1551,D2,2011,2011-07-15,Frankfurt FSV,Union Berlin,1,1,D,20
3,1550,D2,2011,2011-07-16,Erzgebirge Aue,Aachen,1,0,H,19
4,1678,D2,2011,2011-07-16,St Pauli,Ingolstadt,2,0,H,36


In [6]:
df = PandaHandler.win_loss_draw(df)
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,tot_home_goals,HomeWin,AwayWin,tot_home_win,HomeLoss,AwayLoss,tot_home_loss,HomeDraw,AwayDraw,tot_home_draw
0,1133,D2,2011,2011-07-15,Cottbus,Dresden,2,1,H,18,1,0,8,0,1,15,0,0,11
1,1167,D2,2011,2011-07-15,Greuther Furth,Ein Frankfurt,2,3,A,46,0,1,20,1,0,4,0,0,10
2,1551,D2,2011,2011-07-15,Frankfurt FSV,Union Berlin,1,1,D,20,0,0,7,0,0,13,1,1,14
3,1550,D2,2011,2011-07-16,Erzgebirge Aue,Aachen,1,0,H,19,1,0,8,0,1,15,0,0,11
4,1678,D2,2011,2011-07-16,St Pauli,Ingolstadt,2,0,H,36,1,0,18,0,1,8,0,0,8


In [7]:
raindata = RainData(df.iloc[0].Date, df.iloc[-1].Date)

In [8]:
raindf = raindata.get_rain_df(api_key)

In [9]:
raindf.head(10)

,Date,Rain
0,2011-07-15,0
1,2011-07-16,0
2,2011-07-17,1
3,2011-07-18,1
4,2011-07-19,0
5,2011-07-20,0
6,2011-07-21,1
7,2011-07-22,1
8,2011-07-23,1
9,2011-07-24,0


In [10]:
raindf.to_pickle('pickled_rain_df.pkl')

In [11]:
raindf = pd.read_pickle('pickled_rain_df.pkl')

In [12]:
raindf.head()

,Date,Rain
0,2011-07-15,0
1,2011-07-16,0
2,2011-07-17,1
3,2011-07-18,1
4,2011-07-19,0


In [13]:
df_all_data = PandaHandler.rain_results(df, raindf)
df_all_data.head(10)

,Season,GoalsScored,Wins,Losses,Draws,RainGames,RainWins,NonRainWins,RainWin%,NonRainWin%,%ChangeWinWithRain
Team,,,,,,,,,,,
Aachen,2011,15,6,15,13,12.0,2,4,0.166667,-0.333333,-1.500000
Augsburg,2011,20,8,12,14,11.0,4,4,0.363636,-0.363636,-2.000000
Bayern Munich,2011,49,23,7,4,11.0,8,15,0.727273,-1.363636,-1.533333
Bochum,2011,23,10,17,7,10.0,3,7,0.300000,-0.700000,-1.428571
Braunschweig,2011,21,10,9,15,8.0,3,7,0.375000,-0.875000,-1.428571
Cottbus,2011,18,8,15,11,9.0,2,6,0.222222,-0.666667,-1.333333
Dortmund,2011,44,25,3,6,10.0,6,19,0.600000,-1.900000,-1.315789
Dresden,2011,30,12,13,9,8.0,3,9,0.375000,-1.125000,-1.333333
Duisburg,2011,23,10,15,9,15.0,4,6,0.266667,-0.400000,-1.666667


In [15]:
season = str(int(df_all_data.iloc[0].Season))
cols = ['Wins', 'Losses', 'Draws']
bar_x = [season+'_Wins', season+'_Losses', season+'_Draws']

for i in range(len(df_all_data)):
    bar_y = list(df_all_data[cols].iloc[i])
    team_name = df_all_data.index[i]
    plt.bar(x=bar_x, height=bar_y)
    plt.title(team_name)
    plt.savefig('hist_images/{}.png'.format(team_name))
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [17]:
df_all_data['graph'] = [imread('hist_images/{}.png'.format(team_name)).tolist() for team_name in df_all_data.index]
df_all_data.head()

,Season,GoalsScored,Wins,Losses,Draws,RainGames,RainWins,NonRainWins,RainWin%,NonRainWin%,%ChangeWinWithRain,graph
Team,,,,,,,,,,,,
Aachen,2011,15,6,15,13,12.0,2,4,0.166667,-0.333333,-1.500000,"[[[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], ..."
Augsburg,2011,20,8,12,14,11.0,4,4,0.363636,-0.363636,-2.000000,"[[[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], ..."
Bayern Munich,2011,49,23,7,4,11.0,8,15,0.727273,-1.363636,-1.533333,"[[[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], ..."
Bochum,2011,23,10,17,7,10.0,3,7,0.300000,-0.700000,-1.428571,"[[[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], ..."
Braunschweig,2011,21,10,9,15,8.0,3,7,0.375000,-0.875000,-1.428571,"[[[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], ..."


### Mongo DB

Please ensure that your Mongo Database has been opened through the terminal for smooth operation.

In [18]:
client = MongoHandler("mongodb://127.0.0.1:27017/")

In [19]:
collection = client.make_collection('team_stats_db', 'team_stats_collection')
collection

Now in the team_stats_collection collection in the team_stats_db database


Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'team_stats_db'), 'team_stats_collection')

In [20]:
client.clear_collection('team_stats_db','team_stats_collection')

Collection team_stats_collection cleared


In [22]:
results = collection.insert_many(MongoHandler.list_of_dicts(df_all_data))
results.inserted_ids

[ObjectId('5d1a35f1a51d144b174d839f'),
 ObjectId('5d1a35f1a51d144b174d83a0'),
 ObjectId('5d1a35f1a51d144b174d83a1'),
 ObjectId('5d1a35f1a51d144b174d83a2'),
 ObjectId('5d1a35f1a51d144b174d83a3'),
 ObjectId('5d1a35f1a51d144b174d83a4'),
 ObjectId('5d1a35f1a51d144b174d83a5'),
 ObjectId('5d1a35f1a51d144b174d83a6'),
 ObjectId('5d1a35f1a51d144b174d83a7'),
 ObjectId('5d1a35f1a51d144b174d83a8'),
 ObjectId('5d1a35f1a51d144b174d83a9'),
 ObjectId('5d1a35f1a51d144b174d83aa'),
 ObjectId('5d1a35f1a51d144b174d83ab'),
 ObjectId('5d1a35f1a51d144b174d83ac'),
 ObjectId('5d1a35f1a51d144b174d83ad'),
 ObjectId('5d1a35f1a51d144b174d83ae'),
 ObjectId('5d1a35f1a51d144b174d83af'),
 ObjectId('5d1a35f1a51d144b174d83b0'),
 ObjectId('5d1a35f1a51d144b174d83b1'),
 ObjectId('5d1a35f1a51d144b174d83b2'),
 ObjectId('5d1a35f1a51d144b174d83b3'),
 ObjectId('5d1a35f1a51d144b174d83b4'),
 ObjectId('5d1a35f1a51d144b174d83b5'),
 ObjectId('5d1a35f1a51d144b174d83b6'),
 ObjectId('5d1a35f1a51d144b174d83b7'),
 ObjectId('5d1a35f1a51d14

In [23]:
query = client.query_db('team_stats_db', 'team_stats_collection')
for item in query:
    print(item)

{'name': 'Aachen', 'Season': 2011, 'GoalsScored': 15, 'Wins': 6, 'Losses': 15, 'Draws': 13, 'RainGames': 12, 'RainWins': 2, 'NonRainWins': 4, 'RainWin%': 0.1667, 'NonRainWin%': -0.3333, '%ChangeWinWithRain': -1.5}
{'name': 'Augsburg', 'Season': 2011, 'GoalsScored': 20, 'Wins': 8, 'Losses': 12, 'Draws': 14, 'RainGames': 11, 'RainWins': 4, 'NonRainWins': 4, 'RainWin%': 0.3636, 'NonRainWin%': -0.3636, '%ChangeWinWithRain': -2.0}
{'name': 'Bayern Munich', 'Season': 2011, 'GoalsScored': 49, 'Wins': 23, 'Losses': 7, 'Draws': 4, 'RainGames': 11, 'RainWins': 8, 'NonRainWins': 15, 'RainWin%': 0.7273, 'NonRainWin%': -1.3636, '%ChangeWinWithRain': -1.5333}
{'name': 'Bochum', 'Season': 2011, 'GoalsScored': 23, 'Wins': 10, 'Losses': 17, 'Draws': 7, 'RainGames': 10, 'RainWins': 3, 'NonRainWins': 7, 'RainWin%': 0.3, 'NonRainWin%': -0.7, '%ChangeWinWithRain': -1.4286}
{'name': 'Braunschweig', 'Season': 2011, 'GoalsScored': 21, 'Wins': 10, 'Losses': 9, 'Draws': 15, 'RainGames': 8, 'RainWins': 3, 'NonRa

# Working Space

In [ ]:
# The name of the team
# The total number of goals scored by the team during the 2011 season
# The total number of wins the team earned during the 2011 season
# A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
# The team's win percentage on days where it was raining during games in the 2011 season.

In [ ]:
berlin_coordinates = 52.5200, 13.4050

In [ ]:
from PIL import Image
from IPython.display import HTML

df_goals_wins['graph'] = ['<img src="hist_images/{}.png"/>'.format(team_name) for team_name in df_goals_wins.index]
HTML(df_goals_wins.head().to_html(escape=False))